In [1]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero\\research'

In [2]:
import os

In [3]:
os.chdir('../')

In [4]:
%pwd

'c:\\Users\\Kshitij\\Downloads\\DPDZero\\DPDZero'

In [5]:
import pandas as pd

In [9]:
call_logs = pd.read_csv('artifacts/data_ingestion/call_logs.csv')
agent_roster = pd.read_csv('artifacts/data_ingestion/agent_roster.csv')
disposition_summary = pd.read_csv('artifacts/data_ingestion/disposition_summary.csv')

In [16]:
import pandas as pd

# Load the data
call_logs = pd.read_csv('artifacts/data_ingestion/call_logs.csv')
agent_roster = pd.read_csv('artifacts/data_ingestion/agent_roster.csv')
disposition_summary = pd.read_csv('artifacts/data_ingestion/disposition_summary.csv')

# Convert call_date to datetime in both relevant DataFrames
call_logs['call_date'] = pd.to_datetime(call_logs['call_date'])
disposition_summary['call_date'] = pd.to_datetime(disposition_summary['call_date'])

# First, merge agent_roster with disposition_summary to get login information
agent_login = pd.merge(
    agent_roster,
    disposition_summary,
    on=['agent_id', 'org_id'],
    how='left'
)

# Now merge with call logs on agent_id, org_id, and call_date
merged_df = pd.merge(
    agent_login,
    call_logs,
    on=['agent_id', 'org_id', 'call_date'],
    how='left'
)

merged_df.head(300)

,agent_id,users_first_name,users_last_name,users_office_location,org_id,call_date,login_time,call_id,installment_id,status,duration,created_ts
0,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,C8508,L1636,completed,1.42,2025-04-28T02:30:00
1,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,C4164,L1408,failed,5.44,2025-04-28T09:30:00
2,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,C1757,L1250,no_answer,7.89,2025-04-28T20:51:00
3,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,C2746,L1797,no_answer,1.71,2025-04-28T07:28:00
4,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,C6763,L1305,no_answer,13.95,2025-04-28T14:18:00
...,...,...,...,...,...,...,...,...,...,...,...,...
295,A013,AgentFirst13,AgentLast13,Delhi,O1,2025-04-28,10:01,C5386,L1067,connected,0.41,2025-04-28T19:21:00
296,A013,AgentFirst13,AgentLast13,Delhi,O1,2025-04-28,10:01,C6930,L1224,connected,2.81,2025-04-28T17:23:00
297,A013,AgentFirst13,AgentLast13,Delhi,O1,2025-04-28,10:01,C1960,L1168,no_answer,9.97,2025-04-28T00:04:00
298,A013,AgentFirst13,AgentLast13,Delhi,O1,2025-04-28,10:01,C5345,L1948,no_answer,4.74,2025-04-28T09:44:00


In [ ]:
import pandas as pd

# Load the data
call_logs = pd.read_csv('artifacts/data_ingestion/call_logs.csv')
agent_roster = pd.read_csv('artifacts/data_ingestion/agent_roster.csv')
disposition_summary = pd.read_csv('artifacts/data_ingestion/disposition_summary.csv')

# Convert call_date to datetime in both relevant DataFrames
call_logs['call_date'] = pd.to_datetime(call_logs['call_date'])
disposition_summary['call_date'] = pd.to_datetime(disposition_summary['call_date'])

# First, merge agent_roster with disposition_summary to get login information
agent_login = pd.merge(
    agent_roster,
    disposition_summary,
    on=['agent_id', 'org_id'],
    how='left'
)

# Now merge with call logs on agent_id, org_id, and call_date
merged_df = pd.merge(
    agent_login,
    call_logs,
    on=['agent_id', 'org_id', 'call_date'],
    how='left'
)



# Calculate the required metrics
def calculate_metrics(group):
    total_calls = group['call_id'].nunique()
    unique_loans = group['installment_id'].nunique()
    
    completed_calls = group[group['status'] == 'completed']['call_id'].nunique()
    connect_rate = completed_calls / total_calls if total_calls > 0 else 0
    
    avg_duration = group['duration'].mean() if total_calls > 0 else 0
    presence = 1 if pd.notna(group['login_time'].iloc[0]) else 0
    
    return pd.Series({
        'total_calls': total_calls,
        'unique_loans_contacted': unique_loans,
        'connect_rate': connect_rate,
        'avg_call_duration': avg_duration,
        'presence': presence,
        'login_time': group['login_time'].iloc[0] if pd.notna(group['login_time'].iloc[0]) else 'Not Logged In'
    })

# Group by agent_id and call_date to calculate metrics
agent_performance = merged_df.groupby(['agent_id', 'users_first_name', 'users_last_name', 
                                    'users_office_location', 'org_id', 'call_date']).apply(calculate_metrics).reset_index()

# Reorder columns for better readability
agent_performance = agent_performance[[
    'agent_id', 'users_first_name', 'users_last_name', 'users_office_location', 'org_id',
    'call_date', 'login_time', 'presence', 'total_calls', 'unique_loans_contacted',
    'connect_rate', 'avg_call_duration'
]]

# Convert connect_rate to percentage for better readability
agent_performance['connect_rate'] = agent_performance['connect_rate'].apply(lambda x: f"{x:.2%}")

# Display the results
agent_performance.head(21)

C:\Users\Kshitij\AppData\Local\Temp\ipykernel_18792\67209502.py:50: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  'users_office_location', 'org_id', 'call_date']).apply(calculate_metrics).reset_index()


,agent_id,users_first_name,users_last_name,users_office_location,org_id,call_date,login_time,presence,total_calls,unique_loans_contacted,connect_rate,avg_call_duration
0,A001,AgentFirst1,AgentLast1,Bangalore,O1,2025-04-28,11:58,1,20,20,10.00%,6.843000
1,A002,AgentFirst2,AgentLast2,Delhi,O1,2025-04-28,10:05,1,23,23,13.04%,7.827826
2,A003,AgentFirst3,AgentLast3,Mumbai,O1,2025-04-28,10:24,1,21,21,38.10%,7.169524
3,A004,AgentFirst4,AgentLast4,Bangalore,O3,2025-04-28,8:13,1,27,27,14.81%,8.056667
4,A005,AgentFirst5,AgentLast5,Bangalore,O3,2025-04-28,Not Logged In,0,29,28,13.79%,7.059310
5,A006,AgentFirst6,AgentLast6,Chennai,O1,2025-04-28,11:29,1,16,16,12.50%,8.837500
6,A007,AgentFirst7,AgentLast7,Bangalore,O1,2025-04-28,8:36,1,24,23,20.83%,7.040417
7,A008,AgentFirst8,AgentLast8,Mumbai,O1,2025-04-28,9:31,1,28,29,28.57%,6.603103
8,A009,AgentFirst9,AgentLast9,Bangalore,O3,2025-04-28,11:46,1,26,25,26.92%,7.387308
9,A010,AgentFirst10,AgentLast10,Mumbai,O3,2025-04-28,11:11,1,38,38,18.42%,8.078947


In [17]:
from dataclasses import dataclass
from pathlib import Path


@dataclass(frozen=True)
class DataMergingConfig:
    root_dir: Path
    data_dir: Path
    merged_data_dir: Path

In [18]:
from mlProject.constants import *
from mlProject.utils.common import read_yaml,create_directories

In [19]:
class ConfigurationManager:
    def __init__(
        self,
        config_filepath = CONFIG_FILE_PATH,
        params_filepath = PARAMS_FILE_PATH,
        schema_filepath = SCHEMA_FILE_PATH):

        self.config = read_yaml(config_filepath)
        self.params = read_yaml(params_filepath)
        self.schema = read_yaml(schema_filepath)

        create_directories([self.config.artifacts_root])


    
    def get_data_merging_config(self) -> DataMergingConfig:
        config = self.config.data_merging

        create_directories([config.root_dir])

        data_merging_config = DataMergingConfig(
            root_dir = config.root_dir,
            data_dir = config.data_dir,
            merged_data_dir = config.merged_data_dir
        )
        return data_merging_config

In [20]:
import pandas as pd
import os
import urllib.request as request
import zipfile
from mlProject import logger
from mlProject.utils.common import get_size

In [26]:
class DataMerging:
    def __init__(self, config: DataMergingConfig):
        self.config = config

    def merge_data(self):
        call_logs = pd.read_csv(self.config.data_dir + '/call_logs.csv')
        agent_roster = pd.read_csv(self.config.data_dir + '/agent_roster.csv')
        disposition_summary = pd.read_csv(self.config.data_dir + '/disposition_summary.csv')

        call_logs['call_date'] = pd.to_datetime(call_logs['call_date'])
        disposition_summary['call_date'] = pd.to_datetime(disposition_summary['call_date'])

        agent_login = pd.merge(
            agent_roster,
            disposition_summary,
            on=['agent_id', 'org_id'],
            how='left'
        )

        merged_df = pd.merge(
            agent_login,
            call_logs,
            on=['agent_id', 'org_id', 'call_date'],
            how='left'
        )
        merged_df.to_csv(self.config.merged_data_dir, index=False)
        
        logger.info(f"Data merged and saved to {self.config.merged_data_dir}")
        

In [27]:
try:
    config = ConfigurationManager()
    data_merging_config = config.get_data_merging_config()
    data_merging = DataMerging(config=data_merging_config)
    data_merging.merge_data()
except Exception as e:
    raise e


[2025-05-03 19:16:08,190: INFO: common: yaml file: config\config.yaml loaded successfully]
[2025-05-03 19:16:08,196: INFO: common: yaml file: params.yaml loaded successfully]
[2025-05-03 19:16:08,200: INFO: common: yaml file: schema.yaml loaded successfully]
[2025-05-03 19:16:08,204: INFO: common: created directory at: artifacts]
[2025-05-03 19:16:08,207: INFO: common: created directory at: artifacts/data_merging]
[2025-05-03 19:16:08,254: INFO: 3098861574: Data merged and saved to artifacts/data_merging/merged_data.csv]
